In [12]:
from __future__ import print_function
import os, sys
from pprint import pprint
import json
import argparse
import numpy as np
import pickle
from collections import Counter
import time
import time
import numpy as np
from submodlib.helper import create_kernel
from submodlib.functions.facilityLocationMutualInformation import (
    FacilityLocationMutualInformationFunction,
)
from submodlib.functions.facilityLocationVariantMutualInformation import (
    FacilityLocationVariantMutualInformationFunction,
)
from submodlib.functions.graphCutMutualInformation import (
    GraphCutMutualInformationFunction,
)
from submodlib.functions.logDeterminantMutualInformation import (
    LogDeterminantMutualInformationFunction,
)


In [13]:
def budget(budget_size):
    return int(4.92 * budget_size)


In [14]:
def compute_subset(
    greedyList,
    config_dict,
    # dirs,
    # base_dir,
    # ground_list,
    # ground_accent_features,
    # ground_content_features,
    # ground_features_Y,
    # query_list,
    # query_accent_features,
    # test_accent_features,
    # greedyList,
    # budget_size,
    # target_size,
    # accent,
    # fxn,
    # accent_similarity,
    # content_similarity,
    # accent_feature_type,
    # content_feature_type,
    # query_content_features,
    # base_eta,
    # etaScale,
):
    query_dir = config_dict["query_dir"]
    base_dir = config_dict["base_dir"]
    budget_size = config_dict["budget_size"]
    target_size = config_dict["target_size"]
    fxn = config_dict["fxn"]
    accent_feature_type = config_dict["accent_feature_type"]
    content_feature_type = config_dict["content_feature_type"]
    etaScale = config_dict["etaScale"]
    accent_similarity = config_dict["accent_similarity"]
    content_similarity = config_dict["content_similarity"]
    accent = config_dict["accent"]
    ground_list = config_dict["ground_list"]
    data_kernel_type=config_dict["data_kernel_type"]
    query_data_kernel_type=config_dict["query_data_kernel_type"]
    query_query_kernel_type=config_dict["query_query_kernel_type"]

    output_dir = os.path.join(
        base_dir,
        query_dir,
        "all",
        f"budget_{budget_size}",
        f"target_{target_size}",
        f"{fxn}_etaScale_{round(etaScale, 5)}",
        f"accent_{accent_feature_type}",
        f"content_{content_feature_type}",
        f"kernel_g={data_kernel_type}_gq={query_data_kernel_type}_qq={query_query_kernel_type}"
        f"accent_{accent_similarity}",
        f"content_{content_similarity}",
    )
    # os.makedirs(output_dir, exist_ok=True)

    all_indices = [j[0] for j in greedyList]
    all_gains = [j[1] for j in greedyList]

    print("*******all_indices ******")
    print(all_indices)

    print("******all_gains*********")
    print(all_gains)

    total_duration, index = 0, 0
    while total_duration + ground_list[all_indices[index]]["duration"] <= budget(
        budget_size
    ):
        total_duration += ground_list[all_indices[index]]["duration"]
        index += 1
    #         print(index, all_indices[index], len(ground_list))
    total_count = index
    selected_indices = all_indices[:index]
    selected_gains = all_gains[:index]
    print("*******selected_indices ******")
    print(selected_indices)

    print("******selected_gains*********")
    print(selected_gains)
    selected_list = [ground_list[j] for j in selected_indices]

    train_list = selected_list

    accent_sample_count, accent_sample_duration = 0, 0
    for item in selected_list:
        if item["audio_filepath"].split("/")[-4] == accent:
            accent_sample_count += 1
            accent_sample_duration += item["duration"]
    total_selections = Counter(
        [item["audio_filepath"].split("/")[-4] for item in selected_list]
    )

    # with open(f"{output_dir}/train.json", "w") as f:
    #     for line in train_list:
    #         f.write("{}\n".format(json.dumps(line)))

    # plots(dirs, run_dir, ground_features, ground_features_Y, query_features, test_features, selected_indices, selected_gains, fxn)

    print("\n subset computed .... \n")
    stats = "total selections: " + str(total_count)
    stats += "\ntotal duration: " + str(total_duration)
    stats += "\naccented selections: " + str(accent_sample_count)
    stats += "\naccented duration: " + str(accent_sample_duration)
    stats += "\nall selections: " + str(total_selections)
    print(stats)
    # with open(output_dir + "/stats.txt", "w") as f:
    #     f.write(stats)


In [15]:
def generate_greedyList(
    config_dict,
):
    fxn = config_dict["fxn"]
    ground_kernel = config_dict["ground_kernel"]
    query_kernel = config_dict["query_kernel"]
    query_query_kernel = config_dict["query_query_kernel"]
    etaScale = config_dict["etaScale"]
    budget_size = config_dict["budget_size"]
    print(f"\ncreating {fxn} object\n")
    if fxn == "FL1MI":
        obj1 = FacilityLocationMutualInformationFunction(
            n=len(ground_kernel),
            num_queries=query_kernel.shape[1],
            query_sijs=query_kernel,
            data_sijs=ground_kernel,
            magnificationEta=etaScale,
        )
    elif fxn == "LogDMI":
        obj1 = LogDeterminantMutualInformationFunction(
            n=len(ground_kernel),
            num_queries=query_kernel.shape[1],
            lambdaVal=1,  # should this also be tuned?
            query_sijs=query_kernel,
            data_sijs=ground_kernel,
            query_query_sijs=query_query_kernel,
            magnificationEta=etaScale,
        )
    else:
        print("\n\n\n............... ERROR not a valid FUNCTION ............\n\n\n")
        exit()
    print(f"\n{fxn} object created\n")
    print("\ngenerating greedyList...\n")
    greedyList = obj1.maximize(
        budget=3 * budget_size,
        optimizer="LazyGreedy",
        stopIfZeroGain=False,
        stopIfNegativeGain=False,
        epsilon=0.1,
        verbose=False,
        show_progress=True,
    )
    print("\n.... greedyList generated ... \n")
    return greedyList


In [16]:
def load_features(file_dir, feature_type):
    features = []
    file_name = file_dir.split("/")[-1]
    par_dir = "/".join(file_dir.split("/")[:-1]) + f"/{feature_type}/"
    with open(par_dir + file_name.replace(".json", f"_{feature_type}.file"), "rb") as f:
        while True:
            try:
                features.append(pickle.load(f))
            except EOFError:
                break
    #             if features[-1].shape[1]<39:
    #                 print(features[-1].shape)
    #                 print(file_dir, "bad feature file")
    #                 return np.array([[]])
    #                 sys.exit()
    features = np.concatenate(features, axis=0)
    print(features.shape)
    return features


In [17]:
def select_features(accent_features, content_features, reqd_type):
    if(reqd_type == "accent"):
        return accent_features
    elif(reqd_type == "content"):
        return content_features
    else:
        return ValueError(
            f"""
            Reqd_type should be either `accent` or `content`. 
            But `{reqd_type}` is given.
            """
        )

def select_metric(accent_sim, content_sim, reqd_type):
    if(reqd_type == "accent"):
        return accent_sim
    elif(reqd_type == "content"):
        return content_sim
    else:
        return ValueError(
            f"""
            Reqd_type should be either `accent` or `content`. 
            But `{reqd_type}` is given.
            """
        )

In [18]:
def preprocess(config_dict):
    dirs = [
        "kannada_male_english",
        "malayalam_male_english",
        "rajasthani_male_english",
        "hindi_male_english",
        "tamil_male_english",
        "gujarati_female_english",
        "manipuri_female_english",
        "assamese_female_english",
    ]

    target_size = config_dict["target_size"]

    query_dir = f"{config_dict['accent']}/"

    # Query List and # Query Features
    query_file_path = os.path.join(config_dict["base_dir"], query_dir, "seed.json")
    query_list = [json.loads(line.strip()) for line in open(query_file_path)][
        :target_size
    ]

    query_accent_features = load_features(
        query_file_path, config_dict["accent_feature_type"]
    )[:target_size]
    query_content_features = load_features(
        query_file_path, config_dict["content_feature_type"]
    )[:target_size]

    # Ground List and Ground Accent Features, Ground Content Features
    ground_list, ground_list_Y, ground_accent_features, ground_content_features = (
        [],
        [],
        [],
        [],
    )
    for i, _dir in enumerate(dirs):
        selection_file_path = os.path.join(
            config_dict["base_dir"], _dir, "selection.json"
        )
        selection_file_list = [
            json.loads(line.strip()) for line in open(selection_file_path)
        ]
        ground_list.extend(selection_file_list)

        ground_accent_features.append(
            load_features(selection_file_path, config_dict["accent_feature_type"])
        )
        ground_content_features.append(
            load_features(selection_file_path, config_dict["content_feature_type"])
        )

        ground_list_Y.extend([i] * len(selection_file_list))

    ground_accent_features = np.concatenate(ground_accent_features, axis=0)
    ground_content_features = np.concatenate(ground_content_features, axis=0)
    ground_features_Y = np.asarray(ground_list_Y).reshape(-1, 1)

    ### test file
    ## We only need test_accent_features
    ## But for now we are loading both
    # test_file_path = os.path.join(config_dict['base_dir'], query_dir, "test.json")
    # test_list = [json.loads(line.strip()) for line in open(test_file_path)]
    # test_accent_features = load_features(test_file_path, config_dict['accent_feature_type'])
    # test_content_features = load_features(test_file_path, config_dict['content_feature_type'])

    print(
        f"Ground stats",
        f"Length = {len(ground_list)}",
        f"Accent features shape = {ground_accent_features.shape}",
        f"Content features shape = {ground_content_features.shape}",
        sep="\n"
    )
    print("\n\n")
    print(
        f"Query stats",
        f"Length = {len(query_list)}",
        f"Accent features shape = {query_accent_features.shape}",
        f"Content features shape = {query_content_features.shape}",
        sep = "\n"
    )
    # print(
    #     f"Test stats",
    #     f"Length = {len(test_list)}",
    #     f"Accent features shape = {test_accent_features.shape}",
    # )
    # print(len(query_list), query_features.shape)
    # print(len(test_list), test_features.shape)

    print("creating kernels ....")
    t1 = time.time()
    print(f"content_similarity is {config_dict['content_similarity']}")
    ground_kernel = config_dict["ground_ground_wt"] * create_kernel(
        select_features(
            accent_features=ground_accent_features, 
            content_features = ground_content_features, 
            reqd_type = config_dict["data_kernel_type"]
        ), 
        metric=select_metric(
            accent_sim=config_dict["accent_similarity"],
            content_sim=config_dict["content_similarity"],
            reqd_type=config_dict["data_kernel_type"]
        ), 
        mode="dense"
    )
    query_kernel = create_kernel(
        select_features(
            accent_features = query_accent_features,
            content_features = query_content_features,
            reqd_type = config_dict["query_data_kernel_type"]
        ),
        # query_accent_features,
        metric=select_metric(
            accent_sim=config_dict["accent_similarity"],
            content_sim=config_dict["content_similarity"],
            reqd_type=config_dict["query_data_kernel_type"]
        ),
        mode="dense",
        X_rep=select_features(
            accent_features=ground_accent_features, 
            content_features = ground_content_features, 
            reqd_type = config_dict["query_data_kernel_type"]
        ),
    )
    query_query_kernel = create_kernel(
        select_features(
            accent_features = query_accent_features,
            content_features = query_content_features,
            reqd_type = config_dict["query_query_kernel_type"]
        ),
        metric=select_metric(
            accent_sim=config_dict["accent_similarity"],
            content_sim=config_dict["content_similarity"],
            reqd_type=config_dict["query_query_kernel_type"]
        ),
        mode="dense",
    )

    print("************ ground kernel*************")
    print(ground_kernel)
    print("************ query kernel*************")
    print(query_kernel)
    print("************ query query kernel*************")
    print(query_query_kernel)
    t2 = time.time()
    print("kernel creation done ....", t2 - t1)

    print("ground_kernel: ", ground_kernel.shape)
    print("query_kernel: ", query_kernel.shape)
    print("query_query_kernel: ", query_query_kernel.shape)

    return {
            "ground_kernel": ground_kernel,
            "query_kernel": query_kernel,
            "query_query_kernel": query_query_kernel,
            "query_dir": query_dir,
            "ground_list": ground_list,
        }
        # dirs,
        # query_dir,
        # ground_list,
        # ground_accent_features,
        # ground_content_features,
        # ground_features_Y,
        # query_list,
        # query_accent_features,
        # test_accent_features,
        # query_content_features,


In [19]:
def do_expt(**kwargs):
    # parser = argparse.ArgumentParser(description="TSS input")
    # parser.add_argument("--budget", type=int, help="budget")
    # parser.add_argument("--target", type=int, help="target")
    # parser.add_argument("--eta_scale", type=float, help="scale on value")
    # parser.add_argument(
    #     "--accent_similarity", type=str, help="accent_similarity metric"
    # )
    # parser.add_argument(
    #     "--content_similarity", type=str, help="content_similarity_metric"
    # )
    # parser.add_argument("--fxn", type=str, help="function")
    # parser.add_argument("--accent", type=str, help="query set")
    # parser.add_argument(
    #     "--accent_feature_type", type=str, help="which accent feature space"
    # )
    # parser.add_argument(
    #     "--content_feature_type", type=str, help="which content diversity feature space"
    # )

    # args = parser.parse_args()

    # pprint(vars(args))
    args = kwargs
    # pprint(args)

    config_dict = {}
    config_dict["budget_size"] = args["budget"]
    config_dict["target_size"] = args["target"]
    config_dict["etaScale"] = args["eta_scale"]
    config_dict["accent_similarity"] = args["accent_similarity"]
    config_dict["content_similarity"] = args["content_similarity"]
    config_dict["fxn"] = args["fxn"]
    config_dict["accent"] = args["accent"]
    config_dict["accent_feature_type"] = args["accent_feature_type"]
    config_dict["content_feature_type"] = args["content_feature_type"]
    config_dict["base_dir"] = ""
    config_dict["data_kernel_type"] = kwargs["data_kernel_type"]
    config_dict["query_data_kernel_type"] = kwargs["query_data_kernel_type"]
    config_dict["query_query_kernel_type"] = kwargs["query_query_kernel_type"]
    config_dict["ground_ground_wt"] = kwargs["ground_ground_wt"]

    print("***** Printing Experiment configuration settings ******")
    pprint(config_dict)
    print("***** End of Experiment configuration settings ******")

    config_dict.update(preprocess(config_dict))

    greedyList = generate_greedyList(
        config_dict,
        # ground_kernel,
        # query_kernel,
        # query_query_kernel,
        # fxn,
        # budget_size,
        # etaScale,
        # base_eta,
    )
    compute_subset(
        greedyList,
        config_dict,
        # dirs,
        # base_dir,
        # query_dir,
        # ground_list,
        # ground_accent_features,
        # ground_content_features,
        # ground_features_Y,
        # query_list,
        # query_accent_features,
        # test_accent_features,
        # greedyList,
        # budget_size,
        # target_size,
        # accent,
        # fxn,
        # accent_similarity,
        # content_similarity,
        # accent_feature_type,
        # content_feature_type,
        # query_content_features,
        # base_eta,
        # etaScale,
    )


In [20]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.2,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
    "ground_ground_wt": 1,
}

do_expt(**base_args_dict)

***** Printing Experiment configuration settings ******
{'accent': 'kannada_male_english',
 'accent_feature_type': '39_norm',
 'accent_similarity': 'cosine',
 'base_dir': '',
 'budget_size': 150,
 'content_feature_type': 'tf_idf_2gram',
 'content_similarity': 'cosine',
 'data_kernel_type': 'accent',
 'etaScale': 0.2,
 'fxn': 'FL1MI',
 'ground_ground_wt': 1,
 'query_data_kernel_type': 'accent',
 'query_query_kernel_type': 'accent',
 'target_size': 20}
***** End of Experiment configuration settings ******
(50, 39)
(50, 2070)
(3244, 39)
(3244, 2070)
(3491, 39)
(3491, 2070)
(3267, 39)
(3267, 2070)
(3492, 39)
(3492, 2070)
(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine
************ ground kernel***

In [21]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.2,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
    "ground_ground_wt": 1,
}

do_expt(**base_args_dict)

***** Printing Experiment configuration settings ******
{'accent': 'kannada_male_english',
 'accent_feature_type': '39',
 'accent_similarity': 'cosine',
 'base_dir': '',
 'budget_size': 150,
 'content_feature_type': 'tf_idf_2gram',
 'content_similarity': 'cosine',
 'data_kernel_type': 'accent',
 'etaScale': 0.2,
 'fxn': 'FL1MI',
 'ground_ground_wt': 1,
 'query_data_kernel_type': 'accent',
 'query_query_kernel_type': 'accent',
 'target_size': 20}
***** End of Experiment configuration settings ******
(50, 39)
(50, 2070)
(3244, 39)
(3244, 2070)
(3491, 39)
(3491, 2070)
(3267, 39)
(3267, 2070)
(3492, 39)
(3492, 2070)
(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine
************ ground kernel********

In [22]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 1,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
    "ground_ground_wt": 1,
}

do_expt(**base_args_dict)

base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 1,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
    "ground_ground_wt": 1,
}

do_expt(**base_args_dict)

***** Printing Experiment configuration settings ******
{'accent': 'kannada_male_english',
 'accent_feature_type': '39',
 'accent_similarity': 'cosine',
 'base_dir': '',
 'budget_size': 150,
 'content_feature_type': 'tf_idf_2gram',
 'content_similarity': 'cosine',
 'data_kernel_type': 'accent',
 'etaScale': 1,
 'fxn': 'FL1MI',
 'ground_ground_wt': 1,
 'query_data_kernel_type': 'accent',
 'query_query_kernel_type': 'accent',
 'target_size': 20}
***** End of Experiment configuration settings ******
(50, 39)
(50, 2070)
(3244, 39)
(3244, 2070)
(3491, 39)
(3491, 2070)
(3267, 39)
(3267, 2070)
(3492, 39)
(3492, 2070)
(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine
************ ground kernel**********

(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine
************ ground kernel*************
[[1.0000001  0.99806374 0.9969048  ... 0.96295977 0.96142834 0.96202224]
 [0.99806374 1.0000002  0.99877346 ... 0.97030586 0.97151756 0.9695735 ]
 [0.9969048  0.99877346 1.         ... 0.9759629  0.9757933  0.9749282 ]
 ...
 [0.96295977 0.97030586 0.9759629  ... 1.         0.9936852  0.99715984]
 [0.96142834 0.97151756 0.9757933  ... 0.9936852  1.         0.9949801 ]
 [0.96202224 0.9695735  0.9749282  ... 0.99715984 0.9949801  1.        ]]
************ query kernel*************
[[0.99674237 0.9965173  0.9964074  ... 0.9969231  0.99063325 0.9985621 ]
 [0.99809587 0.99864465 0.9982565  ... 0.9961467  0.989068

In [23]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 1,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
    "ground_ground_wt": 100,
}

do_expt(**base_args_dict)

***** Printing Experiment configuration settings ******
{'accent': 'kannada_male_english',
 'accent_feature_type': '39',
 'accent_similarity': 'cosine',
 'base_dir': '',
 'budget_size': 150,
 'content_feature_type': 'tf_idf_2gram',
 'content_similarity': 'cosine',
 'data_kernel_type': 'accent',
 'etaScale': 1,
 'fxn': 'FL1MI',
 'ground_ground_wt': 100,
 'query_data_kernel_type': 'accent',
 'query_query_kernel_type': 'accent',
 'target_size': 20}
***** End of Experiment configuration settings ******
(50, 39)
(50, 2070)
(3244, 39)
(3244, 2070)
(3491, 39)
(3491, 2070)
(3267, 39)
(3267, 2070)
(3492, 39)
(3492, 2070)
(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine
************ ground kernel********

In [10]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.2,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
}

do_expt(**base_args_dict)
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 1,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "accent",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "accent", 
}

do_expt(**base_args_dict)

***** Printing Experiment configuration settings ******
{'accent': 'kannada_male_english',
 'accent_feature_type': '39_norm',
 'accent_similarity': 'cosine',
 'base_dir': '',
 'budget_size': 150,
 'content_feature_type': 'tf_idf_2gram',
 'content_similarity': 'cosine',
 'data_kernel_type': 'accent',
 'etaScale': 0.2,
 'fxn': 'FL1MI',
 'query_data_kernel_type': 'accent',
 'query_query_kernel_type': 'accent',
 'target_size': 20}
***** End of Experiment configuration settings ******
(50, 39)
(50, 2070)
(3244, 39)
(3244, 2070)
(3491, 39)
(3491, 2070)
(3267, 39)
(3267, 2070)
(3492, 39)
(3492, 2070)
(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine


In [13]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.2,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "content",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "content", 
}

do_expt(**base_args_dict)
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.5,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "content",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "content", 
}


do_expt(**base_args_dict)

base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.7,
    "accent_similarity": "cosine",
    "content_similarity": "cosine",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "content",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "content", 
}


do_expt(**base_args_dict)

***** Printing Experiment configuration settings ******
{'accent': 'kannada_male_english',
 'accent_feature_type': '39_norm',
 'accent_similarity': 'cosine',
 'base_dir': '',
 'budget_size': 150,
 'content_feature_type': 'tf_idf_2gram',
 'content_similarity': 'cosine',
 'data_kernel_type': 'content',
 'etaScale': 0.2,
 'fxn': 'FL1MI',
 'query_data_kernel_type': 'accent',
 'query_query_kernel_type': 'content',
 'target_size': 20}
***** End of Experiment configuration settings ******
(50, 39)
(50, 2070)
(3244, 39)
(3244, 2070)
(3491, 39)
(3491, 2070)
(3267, 39)
(3267, 2070)
(3492, 39)
(3492, 2070)
(5183, 39)
(5183, 2070)
(3169, 39)
(3169, 2070)
(6925, 39)
(6925, 2070)
(5765, 39)
(5765, 2070)
Ground stats
Length = 34536
Accent features shape = (34536, 39)
Content features shape = (34536, 2070)



Query stats
Length = 20
Accent features shape = (20, 39)
Content features shape = (20, 2070)
creating kernels ....
content_similarity is cosine
************ ground kernel*************
[[1.       

In [ ]:
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.2,
    "accent_similarity": "euclidean",
    "content_similarity": "euclidean",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "content",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "content", 
}

do_expt(**base_args_dict)
base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.5,
    "accent_similarity": "euclidean",
    "content_similarity": "euclidean",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "content",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "content", 
}


do_expt(**base_args_dict)

base_args_dict = {
    "budget": 150,
    "target": 20,
    "eta_scale": 0.7,
    "accent_similarity": "euclidean",
    "content_similarity": "euclidean",
    "fxn": "FL1MI",
    "accent": "kannada_male_english",
    "accent_feature_type": "39_norm",
    "content_feature_type": "tf_idf_2gram",
    "data_kernel_type": "content",
    "query_data_kernel_type": "accent",
    "query_query_kernel_type": "content", 
}


do_expt(**base_args_dict)